# Generate PDF Report

This example shows how to generate an HTML and PDF report for a specific feature layer. It demonstrates how to create a report that includes summary statistics, photos, maps, and attribute information. In this example we are going to work with a tree inventory layer that was deployed using this [ArcGIS Solution template](https://www.arcgis.com/apps/solutions/index.html?gallery=true&searchTerm=Tree%20Management&solution=vrrh5yvnj1b8ba2tkfab6kf36c2t18np). 

This notebook was originally created as part of DevSummit 2021.

### Connect to our GIS
This notebook uses the ArcGIS API for Python to interact with feature services. So let's import that library and then use it to connect to our GIS so we can fetch the layer containing the trees.

In [1]:
import arcgis

In this example we are going to work with the Trees hosted feature layer that was deployed using this ArcGIS Solution template. Search for this layer in contents and copy and post the item id (found in the url) below where you see `<item-id>` 

In [ ]:
gis = arcgis.gis.GIS("https://www.arcgis.com", "<username>")
item = gis.content.get("<item-id>")
trees_layer = item.layers[0]

### Generate summary statistics
The report should contain some summary statistics about each type of tree. We are going to make a query to fetch, for each type of tree, the:
- count
- average height
- average diameter

In [10]:
statistics = trees_layer.query(
    group_by_fields_for_statistics="COMMONNAME",
    out_statistics=[
        {
            "statisticType": "count",
            "onStatisticField": "COMMONNAME",
            "outStatisticFieldName": "count",
        },
        {
            "statisticType": "avg",
            "onStatisticField": "HEIGHT",
            "outStatisticFieldName": "avg_height",
        },
        {
            "statisticType": "avg",
            "onStatisticField": "DIAMETER",
            "outStatisticFieldName": "avg_diameter",
        },
    ],
).features

### Extract location information from image
The following two functions detail how the EXIF data is parsed to find the latitude, longitude, and date of the photo. The coordinates are stored as Degrees Minutes Seconds in the EXIF data so an additional helper method is used to conver the coordinates to decimal degress.

In [11]:
from PIL import Image, ImageOps, ImageDraw, ImageFont, ExifTags
import base64
import io
import json
import datetime
from PIL.ExifTags import GPSTAGS, TAGS
%matplotlib agg
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from cartopy.io.img_tiles import GoogleTiles
from datetime import datetime as dt

In [12]:
def get_location_data(exif_data):
    gps_info = exif_data.get_ifd(ExifTags.IFD.GPSInfo)
    time = dt.now()
    try:

        lat_degrees = gps_info[ExifTags.GPS.GPSLatitude][0]
        lat_minutes = gps_info[ExifTags.GPS.GPSLatitude][1]
        lat_seconds = gps_info[ExifTags.GPS.GPSLatitude][2]
        lon_degrees = gps_info[ExifTags.GPS.GPSLongitude][0]
        lon_minutes = gps_info[ExifTags.GPS.GPSLongitude][1]
        lon_seconds = gps_info[ExifTags.GPS.GPSLongitude][2]

        lat = float(lat_degrees + lat_minutes / 60 + lat_seconds / 3600)
        lon = float(lon_degrees + lon_minutes / 60 + lon_seconds / 3600)

        if gps_info[ExifTags.GPS.GPSLatitudeRef] == "S":
            lat = -lat
        if gps_info[ExifTags.GPS.GPSLongitudeRef] == "W":
            lon = -lon
        if ExifTags.GPS.GPSTimeStamp in gps_info:
            time = gps_info[ExifTags.GPS.GPSTimeStamp]

    except Exception as e:
        lat = 0
        lon = 0
        time = dt.now()

    return (lat, lon, time)

### Process Attached Images
The report is going to contain a list of all trees and any photos that were captured using ArcGIS Field Maps. Once an image has been downloaded, it's going to be processed using the following function. This fuction will read any EXIF data and orient the image accordingly so it appears correctly in the report. Then it is going to further examine the EXIF data to extract the location and time of when the photo was taken. It is going to overlay this information in the image and then return a base64 encoded version of the image the will be embedded in the report.

To accomplish this, it uses the [Pillow](https://pillow.readthedocs.io/en/stable/) library which is a very popuplar image process library for Python.

In [13]:
def process_image(input_file):
    image = Image.open(input_file)
    image = ImageOps.exif_transpose(image)
    lat, long, time = get_location_data(image.getexif())
    image_editable = ImageDraw.Draw(image)
    font = ImageFont.truetype("/Library/Fonts/Arial Unicode.ttf", 24)
    image_editable.text(
        (1, 1),
        f'{lat}, {long} at {time.strftime("%Y-%m-%d %H:%M:%S")}',
        (255, 255, 0),
        font=font,
    )
    io_bytes = io.BytesIO()
    image.save(io_bytes, format="JPEG")
    io_bytes.seek(0)
    return base64.b64encode(io_bytes.read()).decode()

### Generate a map image
The report is also going to contain a small map for each tree to show where it is located. The [cartopy](https://scitools.org.uk/cartopy/docs/latest/) libary is used in conjunction with [matpotlib](https://matplotlib.org/) to generate a single image that plots the location on a map using the Esri World Street Map Tile service. The function returns a base64 encoded image that will be embedded into the report.

In [14]:
def create_map(feature):
    fig = plt.figure(figsize=(5, 5))
    url = "https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}.jpg"
    image = GoogleTiles(url=url)
    ax = fig.add_subplot(1, 1, 1, projection=image.crs)
    ax.add_image(image, 15)
    xmin = feature["geometry"]["x"] - 500
    xmax = feature["geometry"]["x"] + 500
    ymin = feature["geometry"]["y"] - 500
    ymax = feature["geometry"]["y"] + 500
    ax.set_extent([xmin, xmax, ymin, ymax], crs=image.crs)
    ax.scatter(
        feature["geometry"]["x"],
        feature["geometry"]["y"],
        color="green",
        linewidth=2,
        marker="o",
    )
    io_bytes = io.BytesIO()
    plt.savefig(io_bytes, format="jpeg", bbox_inches="tight")
    io_bytes.seek(0)
    return base64.b64encode(io_bytes.read()).decode()

### Build a data model for each feature
Now that we have our helper methods all in place, let's query all of the trees. For each tree, we are going to generate a map image to embed into the report. Then we are going to fetch all of the jpeg images associated with the tree. For each of those images, we are going to process it to extract and overlay the location. Each feature will now have a dictionary representing its geometry, a dictionary of its attributes, a list of base64 encoded attachments, and a single base64 encoded image of its location on a map.

In [15]:
trees = [f.as_dict for f in trees_layer.query("1=1").features]
for feature in trees:
    feature["map"] = create_map(feature)
    attachments = trees_layer.attachments.search(
        attachment_types=["image/jpeg", "image/jpg"],
        object_ids=[feature["attributes"]["OBJECTID"]],
    )
    feature["attachments"] = []
    for a in attachments:
        f = trees_layer.attachments.download(
            oid=feature["attributes"]["OBJECTID"], attachment_id=a["ID"]
        )[0]
        image = process_image(f)
        feature["attachments"].append(image)

### Generate the report
In order to build the report, we are using a templating engine called [jinja2](https://jinja.palletsprojects.com/en/2.11.x/). This templating engine is very popular with various Python web frameworks like Flask. An html template file has been created and we are going to use jinja to substitute in our statistics and list of trees. This will result in a complete html string that could be served as a static webpage if desired. 

In [16]:
from jinja2 import Environment, FileSystemLoader

env = Environment(loader=FileSystemLoader("."))
template = env.get_template("report-template.html")
template_variables = {
    "title": item.title,
    "statistics": [s.attributes for s in statistics],
    "features": trees,
}
generated_html = template.render(template_variables)

### Export to PDF
Finally we are going to convert the html to a PDF file using [weasyprint](https://weasyprint.org/). After this completes you should be able to open the generated PDF and view your report.

In [17]:
from weasyprint import HTML

In [18]:
HTML(string=generated_html).write_pdf("report.pdf")

### Summary
This notebook demostrated how to build a custom PDF report by combining the ArcGIS API for Python, and a handful of other open-source tools. By using HTML as a conversion layer, it's very easy to tweak the content and style of the report.